## Start

In [ ]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Source #Partner

### Links: 
___

In [ ]:
# creating a table from s3

query_text = """--sql 
    CREATE TABLE db1.billing_orders_devices_st_partner
    (   
        `billing_account_id` Int64,
        `cost` Float64,
        `count` Int64,
        `created_at` DateTime,
        `device_type` String,
        `device_uuid` String,
        `partner_uuid` String,
        `report_date` Date,
        `service` String,
        `state` String,
        `total` Float64
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/billing_orders_devices_st_partner/year=*/month=*/*.csv','CSVWithNames')
    PARTITION BY billing_account_id
    """

ch.query_run(query_text)

In [ ]:
query_text = """--sql
    CREATE TABLE db1.billing_orders_devices_st_partner_ch
    (   
        `billing_account_id` Int64,
        `cost` Float64,
        `count` Int64,
        `created_at` DateTime,
        `device_type` String,
        `device_uuid` String,
        `partner_uuid` String,
        `report_date` Date,
        `service` String,
        `state` String,
        `total` Float64
    )
    ENGINE = MergeTree()
    ORDER BY partner_uuid
    """

ch.query_run(query_text)

In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.billing_orders_devices_st_partner_mv
    REFRESH EVERY 1 DAY OFFSET 4 HOUR RANDOMIZE FOR 1 HOUR TO db1.billing_orders_devices_st_partner_ch AS
    SELECT
        *
    FROM db1.billing_orders_devices_st_partner
    """

ch.query_run(query_text)

___
## Tools
___
### query


In [ ]:
query_text = """--sql
    SELECT
        *
    FROM db1.billing_orders_devices_st_partner_ch
    ORDER BY report_date desc
    limit 100
    """

ch.query_run(query_text)


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.billing_orders_devices_st_partner_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [ ]:
query_text = """--sql
    DROP TABLE db1.billing_orders_devices_st_partner_mv
    """

ch.query_run(query_text)


### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.billing_orders_devices_st_partner_ch
    """

ch.query_run(query_text)

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.billing_orders_devices_st_partner_mv
"""

ch.query_run(query_text)